In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(777)

# 1.Pytorch로 구현하기

In [11]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

In [21]:
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer = 10
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), # tensor flow 의 경우는 항상 사용으로 설정 되어있다. use_bais=False로 별도로 설정해야 사용하지 않음
    nn.Sigmoid(), 
    nn.Linear(10, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 1, bias=True),
    nn.Sigmoid()  # 이 부분이 수정되었습니다.
)


tensorflow 처럼 ReLU 를 사용하려면 loss function 을 바꿔줘야 한다. \
sigmoid는 0, 1 사이 값 ==> binary cross entropy 사용

```python

model = nn.Sequential(
    nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer = 10
    nn.ReLU(),
    nn.Linear(10, 10, bias=True),
    nn.ReLU(), 
    nn.Linear(10, 10, bias=True),
    nn.ReLU(),
    nn.Linear(10, 1, bias=True),
    # nn.ReLU()  # 마지막 ReLU는 제거
)

## 변경되는 부분BCEWithLogitsLoss를 사용 ##
criterion = torch.nn.BCEWithLogitsLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X).squeeze()  # squeeze를 사용해 출력의 차원을 줄여줍니다.

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())


```

In [22]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [23]:
for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())


0 0.6969952583312988
100 0.6931243538856506
200 0.6931226253509521
300 0.6931207180023193
400 0.6931186318397522
500 0.6931164264678955
600 0.6931140422821045
700 0.6931114196777344
800 0.6931085586547852
900 0.6931054592132568
1000 0.6931020021438599
1100 0.6930981278419495
1200 0.6930938363075256
1300 0.693088948726654
1400 0.6930834650993347
1500 0.6930773258209229
1600 0.6930702924728394
1700 0.6930621862411499
1800 0.6930526494979858
1900 0.6930416226387024
2000 0.6930285692214966
2100 0.6930129528045654
2200 0.6929939985275269
2300 0.6929707527160645
2400 0.6929415464401245
2500 0.6929042935371399
2600 0.6928554773330688
2700 0.6927899122238159
2800 0.6926986575126648
2900 0.6925661563873291
3000 0.6923627853393555
3100 0.6920280456542969
3200 0.6914203763008118
3300 0.6901488304138184
3400 0.6868014335632324
3500 0.6732664108276367
3600 0.5683151483535767
3700 0.1186613216996193
3800 0.0124523239210248
3900 0.00553008122369647
4000 0.003406185656785965
4100 0.0024168938398361206

In [26]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ')
    print(hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ')
    print(predicted.detach().cpu().numpy())
    print('실제값(Y): ')
    print(Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis): 
[[7.2205643e-05]
 [9.9990404e-01]
 [9.9989367e-01]
 [1.4228253e-04]]
모델의 예측값(Predicted): 
[[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y): 
[[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0


# 2. Tensorflow로 구현하기

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 데이터 준비
x_train = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
y_train = np.array([[0],[1],[1],[0]], dtype=np.float32)

# 모델 정의
model = keras.Sequential([
    keras.layers.Dense(units=10, activation='relu', input_shape=(2,)), # 첫 번째 히든 레이어
    keras.layers.Dense(units=10, activation='relu'), # 두 번째 히든 레이어
    keras.layers.Dense(units=10, activation='relu'), # 세 번째 히든 레이어
    keras.layers.Dense(units=1, activation='sigmoid') # 출력 레이어
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.SGD(lr=0.1), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=2000, verbose=0)

# 모델 평가
print(model.evaluate(x_train, y_train))
